<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Практическая работа № 6**  
## **Трансформеры и BERT-вдохновлённые архитектуры для табличных данных и временных рядов**

---

### **1. Цель и задачи работы**

**Цель работы** — формирование у обучающегося системного представления о методологии сравнительного анализа современных архитектур на основе механизма внимания и BERT-парадигмы, применяемых к задачам анализа структурированных данных, включая как табличные датасеты, так и временные ряды. Работа направлена на освоение практических навыков проектирования, настройки, диагностики и интерпретации специализированных трансформерных моделей, построенных после 2020 года, при условии соблюдения строгих требований к предобработке, временной валидации, отсутствию утечки будущего и воспроизводимости результатов в соответствии с передовыми научно-методическими стандартами.

**Основные задачи работы:**  
1. Использовать два типа данных:  
   – **Временной ряд**, обработанный в Практической работе № 1, содержащий не менее 720 последовательных наблюдений, с подтверждённой стационарностью, стабилизированной дисперсией и богатым набором инженерных признаков (лаги, скользящие окна, циклические кодировки);  
   – **Табличный датасет без временной структуры**, содержащий смешанные типы признаков (например, California Housing для регрессии или Adult Income для классификации).  
2. Формализовать задачу машинного обучения с учётом природы данных: для временных рядов — как задачу прогнозирования с соблюдением временного порядка; для табличных данных — как стандартную задачу регрессии или классификации.  
3. Определить архитектуры и гиперпараметры для всех моделей и провести их обучение с использованием корректной валидации: `TimeSeriesSplit` для временных рядов, стратифицированное разбиение — для табличных данных.  
4. Реализовать и сравнить следующие категории моделей:  
   – **Специализированные трансформеры для временных рядов**:  
     • Informer (ProbSparse Attention),  
     • Autoformer (автокорреляция),  
     • FEDformer (частотно-доменный attention),  
     • TimesNet (1D→2D периодическое преобразование),  
     • TSMixer (MLP-смеси по времени и признакам);  
   – **BERT-вдохновлённые модели для временных рядов**:  
     • PatchTST (представление ряда как последовательности «патчей» + Transformer encoder),  
     • TimeBERT (маскированное обучение с временными эмбеддингами),  
     • TS2Vec (контрастивное обучение временных представлений);  
   – **Трансформеры для табличных данных**:  
     • TabTransformer (Transformer над эмбеддингами категориальных признаков),  
     • FT-Transformer (унифицированная токенизация всех признаков).  
5. Обеспечить корректную предобработку: масштабирование числовых признаков, кодирование категориальных, разбиение на патчи или окна, применение маскирования только на обучающей части.  
6. Провести обучение в двух режимах:  
   – **Supervised-only** (для моделей без самообучения);  
   – **Self-supervised pretraining + fine-tuning** (для BERT-подобных моделей).  
7. Оценить качество прогнозов/классификации по метрикам:  
   – Для регрессии: MAE, RMSE, MASE, SMAPE;  
   – Для классификации: Accuracy, F1, AUC-ROC — **на независимой тестовой выборке, не участвовавшей в настройке**.  
8. Проанализировать:  
   – **Вычислительную эффективность**: время обучения, память, масштабируемость по длине последовательности;  
   – **Интерпретируемость**: attention maps, важность патчей, вклад категориальных признаков;  
   – **Эффективность самообучения**: прирост качества при ограниченных labelled данных.  
9. Сравнить все модели между собой и с **лучшими решениями из Практических работ № 3 (классический ML) и № 5 (глубокое обучение без трансформеров)**.  
10. Оценить вычислительную сложность: время обучения, прогнозирования и потребление памяти.  
11. Разработать модульный фреймворк на PyTorch с единым интерфейсом для всех архитектур и типов данных.  
12. Обеспечить воспроизводимость и открытость результатов: опубликовать код, конфигурации, веса, визуализации и аналитический отчёт.

---

### **2. Теоретические предпосылки**

Появление BERT в 2018 году ознаменовало сдвиг от поверхностного к **контекстуальному представлению данных**. Однако его прямое применение к числовым последовательностям методологически некорректно: временные ряды не обладают словарём, грамматикой или семантикой, свойственными естественному языку. Тем не менее, **ключевые идеи BERT** — маскированное моделирование, глубокий encoder, позиционная осведомлённость — могут быть **переосмыслены** для структурированных данных.

Современные архитектуры решают эту задачу по-разному.  
– **PatchTST** трактует временной ряд как последовательность «токенов», где токеном выступает **патч** — непересекающееся окно из k точек. Это позволяет применить стандартный Transformer encoder, но с **улучшенной позиционной кодировкой**, учитывающей как глобальный, так и локальный порядок.  
– **TimeBERT** вводит **богатые временные эмбеддинги**: абсолютное время, относительное смещение, циклические компоненты (день недели, месяц), что даёт модели понимание временной структуры. Обучение ведётся через восстановление маскированных наблюдений — аналог MLM в BERT.  
– **TS2Vec** отказывается от attention в пользу **контрастивного обучения**: близкие по времени сегменты должны иметь схожие представления в скрытом пространстве, что обеспечивает устойчивость к шуму и обобщение.

Для **специализированных трансформеров** характерен отказ от dogmatic self-attention:  
– **Autoformer** заменяет его на **автокорреляцию** — естественный механизм для временных зависимостей;  
– **FEDformer** работает в **частотной области**, выделяя глобальные тренды и сезонности;  
– **TimesNet** преобразует 1D-ряд в 2D-матрицу, чтобы применить **2D-CNN** к обнаруженным периодам;  
– **TSMixer** вообще отказывается от attention, используя **MLP по временным и признаковым измерениям** — простота, скорость, эффективность.

Для **табличных данных** трансформеры применяются через **трансформацию признаков в последовательность**:  
– **TabTransformer** строит attention-взаимодействия только между категориальными признаками, оставляя числовые вне encoder’а;  
– **FT-Transformer** унифицирует все признаки через линейную токенизацию, создавая симметричную архитектуру.

Важно подчеркнуть: **ни одна из этих моделей не использует предобученные веса BERT из Hugging Face**. Все они — **специализированные архитектуры**, обучаемые с нуля на числовых данных. Это принципиальное отличие от наивного «подставить числа вместо слов».

---

### **3. Порядок выполнения работы**

Работа выполняется в несколько последовательных этапов, каждый из которых направлен на достижение конкретных образовательных и исследовательских результатов.

#### **3.1. Использование данных из Практической работы № 1 и дополнительного табличного датасета**

**Задача:** Обеспечить методологическую преемственность и введение разнообразных сценариев.  
**Требования к выполнению:**  
– **Временной ряд**: финальная таблица из Практической работы № 1:  
  • Объём: не менее 720 последовательных наблюдений;  
  • Наличие целевой переменной и не менее пяти дополнительных признаков (включая лаги \( y_{t-1}, y_{t-7}, y_{t-30} \), скользящие средние, циклические кодировки);  
  • Подтверждённая стационарность (результаты ADF/KPSS);  
  • Сохранённые параметры преобразований (например, \( \lambda \) для Бокса–Кокса).  
– **Табличный датасет**: например, `California Housing` (регрессия) или `Adult Income` (классификация):  
  • Содержит числовые и категориальные признаки;  
  • Объём ≥10 000 наблюдений.  
– Все признаки представлены в табличной форме без пропусков на момент обучения.

#### **3.2. Формализация задачи и разделение данных**

**Задача:** Корректно сформулировать задачу машинного обучения с учётом природы данных.  
**Требования к выполнению:**  
– Для временных рядов:  
  • Целевая переменная: \( y_t \); признаки \( X_t \): все доступные лаги и инженерные признаки;  
  • Разделение: тренировочная + валидационная часть — всё до \( T - 90 \); финальный тест — последние 90 наблюдений (только для итоговой оценки);  
  • Для кросс-валидации: `TimeSeriesSplit(n_splits=5, test_size=60)` — без перемешивания.  
– Для табличных данных:  
  • Стандартная задача регрессии/классификации;  
  • Разделение: 80/10/10 с стратификацией по целевой переменной.

#### **3.3. Определение архитектур и настройка моделей**

**Задача:** Привести каждую модель к её оптимальной конфигурации с использованием корректной валидации.  
**Требования к выполнению:**  
– Для всех моделей определяются гиперпараметры на основе рекомендаций литературы:  
  • **Informer**: `factor=5`, `d_model=512`;  
  • **PatchTST**: `patch_len=16`, `stride=16`;  
  • **TabTransformer**: `num_heads=8`, `num_encoder_layers=6`;  
  • **TimesNet**: `top_k=5`, `d_model=128`.  
– Настройка:  
  • Обучение с фиксированным seed;  
  • Оптимизатор: AdamW (lr=1e-4), learning rate scheduling;  
  • Регуляризация: dropout (0.1–0.3), weight decay, gradient clipping;  
  • Остановка: early stopping по val loss (patience=10).  
– Все процедуры выполняются только на тренировочной части с корректной валидацией.  
– Фиксация: оптимальные гиперпараметры, значение метрики на валидации, время обучения.

#### **3.4. Обучение в режимах supervised и self-supervised**

**Задача:** Реализовать корректное обучение BERT-подобных моделей.  
**Требования к выполнению:**  
– Для моделей **без самообучения** (Informer, TSMixer): прямое supervised обучение;  
– Для моделей **с самообучением** (PatchTST, TimeBERT, TS2Vec):  
  • Этап 1: pretraining на всём тренировочном ряду с маскированием 15–25% патчей/точек;  
  • Этап 2: fine-tuning на задачу прогнозирования;  
– Маскирование применяется **только к обучающей части**, без доступа к будущему;  
– Сравнение качества с и без pretraining.

#### **3.5. Построение и настройка моделей**

**Задача:** Реализовать единообразный интерфейс для всех трансформерных моделей.  
**Требования к выполнению:**  
– Все модели реализуются на PyTorch с единым API:  
  • `model.fit(X_train, y_train)`;  
  • `model.predict(X_test)`.  
– Поддержка GPU и CPU;  
– Для временных рядов — вход в формате `(batch, seq_len, features)`;  
– Для табличных — `(batch, num_features)`.  
– Используются официальные или воспроизведённые по статьям реализации.

#### **3.6. Диагностика адекватности моделей**

**Задача:** Убедиться, что модель корректно обучается и не допускает утечки будущего.  
**Требования к выполнению:**  
– **Анализ утечки**: проверка, что маскирование не использует информацию из будущего;  
– **Графики обучения**: train/val loss по эпохам;  
– **Стабильность**: повторный запуск с разными seed — насколько стабильны метрики?  
– **Масштабируемость**: измерение времени обучения при увеличении длины ряда (T = 100, 500, 1000).  
– Диагностика обязательна для всех моделей и обоих типов данных.

#### **3.7. Оценка качества и статистическое сравнение**

**Задача:** Обоснованно сравнить модели по объективным критериям.  
**Требования к выполнению:**  
– Расчёт метрик **на финальной тестовой выборке** для каждого датасета и горизонта (\( h = 1, 7, 30 \)):  
  • Регрессия: MAE, RMSE, MASE, SMAPE;  
  • Классификация: Accuracy, F1, AUC-ROC.  
– Применение **теста Дикболда–Мариано** для попарного сравнения прогнозов топ-5 моделей;  
– Составление сравнительных таблиц:  
  • Модели × датасеты × метрики × время обучения × время прогноза;  
  • Интерпретируемость (attention maps, важность патчей);  
  • Эффективность самообучения (прирост качества).  
– Ранжирование моделей по взвешенной оценке: точность (50%), интерпретируемость (30%), вычислительная эффективность (20%).

#### **3.8. Анализ интерпретируемости моделей**

**Задача:** Оценить способность модели объяснять свои предсказания.  
**Требования к выполнению:**  
– Для **PatchTST/Informer/TabTransformer**: визуализация attention-карт;  
– Для **TimesNet**: анализ активных периодов в CNN-слоях;  
– Для **TimeBERT**: оценка качества восстановления маскированных точек;  
– Для **FT-Transformer**: расчёт важности признаков через attention weights;  
– Проведение **локальной интерпретации**: heatmaps для отдельных прогнозов;  
– Результаты интерпретации включаются в сравнительные таблицы.

#### **3.9. Анализ вычислительной эффективности**

**Задача:** Оценить практическую применимость моделей в условиях ограниченных ресурсов.  
**Требования к выполнению:**  
– Фиксация времени:  
  • Обучение (в секундах),  
  • Прогнозирование (в миллисекундах на наблюдение);  
– Оценка потребления памяти (опционально, через `memory_profiler`);  
– Сравнение сложности: O(N) vs. O(N²) по длине последовательности;  
– Включение столбца «Вычислительная сложность» в сравнительные таблицы: низкая/средняя/высокая.

#### **3.10. Разработка модульного фреймворка**

**Задача:** Создать единый интерфейс для всех архитектур и типов данных.  
**Требования к выполнению:**  
– Единый базовый класс `BaseTransformerModel` с методами `fit()`, `predict()`, `interpret()`;  
– Поддержка переключения между задачами (табличная/временная) через конфигурацию;  
– Автоматическая предобработка признаков (масштабирование, патчинг, эмбеддинги);  
– Экспорт/загрузка весов моделей.

#### **3.11. Публикация результатов исследования**

**Задача:** Обеспечить открытость и воспроизводимость работы.  
**Требования к выполнению:**  
– Для кода и результатов составляется метаописание, включающее:  
  • Список моделей и гиперпараметров;  
  • Оптимальные гиперпараметры и метрики на валидации;  
  • Сравнительные таблицы по всем моделям и датасетам;  
  • Ключевые выводы и рекомендации;  
  • Инструкцию по запуску фреймворка.  
– Все материалы публикуются в публичном репозитории (GitHub/GitLab);  
– Веса моделей и логи обучения — архивированы;  
– Использование открытых лицензий: MIT (код), CC BY 4.0 (данные).

---

### **4. Дополнительные исследовательские задания**

1. Влияние длины патча: как качество PatchTST зависит от \( k = 4, 8, 16, 32 \)?  
2. Обучение с малым числом меток: сравнение supervised-only и self-supervised подходов при 10%, 30%, 50% labelled данных.  
3. Робастность к пропускам: как модели ведут себя при 10–30% пропущенных значений?  
4. Гибридизация: объединение представлений из FT-Transformer и прогнозов LightGBM через стекинг.  
5. Анализ утечки: что происходит, если в маскирующей модели случайно разрешить доступ к будущему?  
6. Сравнение скорости масштабирования: как время обучения растёт с длиной ряда (T = 100, 500, 1000, 2000)?

---

### **5. Требования к отчёту**

Отчёт оформляется в соответствии с ГОСТ 7.32–2017 и должен содержать следующие разделы:  
1. **Введение** — от BERT к специализированным трансформерам; актуальность, обзор литературы, указание на преемственность с Практическими работами № 1, № 3, № 5.  
2. **Методология** — описание данных, архитектур, стратегий обучения, предотвращения утечки будущего, метрик.  
3. **Результаты экспериментов** — сравнительные таблицы, графики прогнозов, attention maps, результаты теста Дикболда–Мариано, диагностика утечки.  
4. **Обсуждение** — критический анализ: attention vs. автокорреляция vs. частотный анализ; эффективность самообучения; сравнение с классическим и глубоким ML.  
5. **Заключение** — итоговые выводы, практические рекомендации по выбору архитектуры в зависимости от объёма данных, горизонта, требований к интерпретируемости и вычислительным ресурсам.  
6. **Список использованных источников** — оформлен в соответствии с ГОСТ Р 7.0.5–2008.  
7. **Приложения** — архитектурные схемы, фрагменты кода, скриншоты визуализаций, таблицы гиперпараметров.  
Отчёт сопровождается ссылками на репозиторий с кодом и опубликованные данные.

---

### **6. Критерии оценивания**

| Оценка | Критерии |
|--------|----------|
| **Отлично** | Полное выполнение всех этапов; обучение ≥7 моделей (включая BERT-подобные); корректная валидация без утечки будущего; углублённая диагностика (attention maps, патчи, эмбеддинги); сравнение с ML/DL; модульный фреймворк; публикация всех материалов; отчёт, соответствующий ГОСТ. |
| **Хорошо** | Выполнение основных этапов; обучение 4–6 моделей; наличие сравнительных таблиц и базовой интерпретации; отсутствие лишь отдельных элементов (например, анализа утечки или сравнения с прошлыми работами). |
| **Удовлетворительно** | Реализованы 2–3 модели (например, только Informer и PatchTST); есть прогноз и метрики, но без диагностики утечки или интерпретации. |
| **Неудовлетворительно** | Применён стандартный BERT из Hugging Face без адаптации; нет корректного разделения данных; отсутствует сравнительный анализ; отчёт не представлен. |
